<a href="https://colab.research.google.com/github/pewtpong/wespace-forest-ml/blob/main/forest_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import os
import numpy as np
import cv2
import argparse
import time
from tqdm import tqdm
from google.colab.patches import cv2_imshow
import glob
import shutil 

In [ ]:
def drawRect(df, image):
  temp_df = df.copy()
  temp_image = image.copy()
  for index, row in temp_df.reset_index().iterrows():
    # print(row["xmin"], row["ymin"], row["xmax"], row["ymax"])
    start_point = (row["xmin"], row["ymin"])
    end_point = (row["xmax"], row["ymax"])
    color = (0, 0, 255)
    thickness = 1
    temp_image = cv2.rectangle(temp_image, start_point, end_point, color, thickness)
  return temp_image


In [ ]:
def rotate_image(image, angle):
  """
  Rotates an image (angle in degrees) and expands image to avoid cropping
  """
  height, width = image.shape[:2]  # image shape has 3 dimensions
  image_center = (width / 2,
                  height / 2)  # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape
  rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)
  # rotation calculates the cos and sin, taking absolutes of those.
  abs_cos = abs(rotation_mat[0, 0])
  abs_sin = abs(rotation_mat[0, 1])
  # find the new width and height bounds
  bound_w = int(height * abs_sin + width * abs_cos)
  bound_h = int(height * abs_cos + width * abs_sin)
  # subtract old image center (bringing image back to origin) and adding the new image center coordinates
  rotation_mat[0, 2] += bound_w / 2 - image_center[0]
  rotation_mat[1, 2] += bound_h / 2 - image_center[1]
  # rotate image with the new bounds and translated rotation matrix
  rotated_mat = cv2.warpAffine(image, rotation_mat, (bound_w, bound_h))
  return rotated_mat

In [ ]:
def rotateBox(image, df, angle, filePath):
  temp_df = df.copy()

  h, w = image.shape[:2] 
  cx, cy = (int(w / 2), int(h / 2))
  outArr = []
  listFilePath = filePath.split("/")[-1].split(".")
  fileName = listFilePath[0] + "_angle_{}".format(angle) + ".{}".format(listFilePath[1])
  for index, row in temp_df.reset_index().iterrows():
    bbox_tuple = [
            (row["xmin"], row["ymin"]),
            (row["xmax"], row["ymax"]),
        ] # put x and y coordinates in tuples, we will iterate through the tuples and perform rotation

    rotated_bbox = []

    for i, coord in enumerate(bbox_tuple):

      M = cv2.getRotationMatrix2D((cx, cy), angle, 1.0)
      cos, sin = abs(M[0, 0]), abs(M[0, 1])
      newW = int((h * sin) + (w * cos))
      newH = int((h * cos) + (w * sin))
      M[0, 2] += (newW / 2) - cx
      M[1, 2] += (newH / 2) - cy
      v = [coord[0], coord[1], 1]
      adjusted_coord = np.dot(M, v)
      rotated_bbox.insert(i, (adjusted_coord[0], adjusted_coord[1]))

    result = [int(x) for t in rotated_bbox for x in t]
    if angle % 90 != 0:
      temp_w = (result[2] - result[0]) // 2
      result[1] = max(0, result[1] - temp_w)
      result[3] = result[3] + temp_w

    result.insert(0, fileName)
    result.append(row['label'])
    outArr.append(result)
  return outArr, fileName
  


In [ ]:
location = "benja"
train_file = "/content/gdrive/MyDrive/ML_purpose/forest/csv/{}/{}_train.csv".format(location, location)

# Combine

## Copy image folder

In [ ]:
src = '/content/gdrive/MyDrive/ML_purpose/forest/{}_raw'.format(location)
dest = '/content/gdrive/MyDrive/ML_purpose/forest/{}_train'.format(location)
    
destination = shutil.copytree(src, dest) 

## Delete unused file in train

In [ ]:
raw_df = pd.read_csv("/content/gdrive/MyDrive/ML_purpose/forest/csv/{}/{}_train.csv".format(location, location))
useful_image = raw_df["image_path"].unique().tolist()
file_list = glob.glob('/content/gdrive/MyDrive/ML_purpose/forest/{}_train/*.png'.format(location))

In [ ]:
angles = [90, 180, 270]

imgRoot = '/content/gdrive/MyDrive/ML_purpose/forest/{}_train/{}'

out_arr = []
out_arr.append(raw_df)
for filePath in file_list:
  fileName = filePath.split("/")[-1]
  if fileName not in useful_image:
    os.remove(filePath)
    continue
  image = cv2.imread(filePath)
  temp_df = raw_df[raw_df["image_path"] == fileName]
  for angle in angles:
    rotated_image = rotate_image(image, angle)
    out, newFileName = rotateBox(image, temp_df, angle, filePath)
    rotated_df = pd.DataFrame(out, columns = ['image_path', 'xmin', 'ymin', 'xmax', 'ymax', 'label'])
    cv2.imwrite(imgRoot.format(location, newFileName), rotated_image)
    out_arr.append(rotated_df)

out_df = pd.concat(out_arr).reset_index()[['image_path', 'xmin', 'ymin', 'xmax', 'ymax', 'label']]
for index, row in out_df.iterrows():
  if row["xmax"] < row["xmin"]:
    row["xmax"], row["xmin"] = row["xmin"], row["xmax"]
  if row["ymax"] < row["ymin"]:
    row["ymax"], row["ymin"] = row["ymin"], row["ymax"]

In [ ]:
out_df = out_df[(out_df["ymax"] > out_df["ymin"]) & (out_df["xmax"] > out_df["xmin"])]

In [ ]:
out_df[out_df["xmax"] <= out_df["xmin"]]

,image_path,xmin,ymin,xmax,ymax,label


In [ ]:
out_df.to_csv("/content/gdrive/MyDrive/ML_purpose/forest/csv/{}/{}_train_full.csv".format(location, location), index=False)